### Cell 1: Chuẩn bị Môi trường và Tải Font Tiếng Việt
Cell này thực hiện các công việc sau:
1. **Import thư viện cần thiết:** `os`, `requests`, `zipfile`, `io`.
2. **Tạo cấu trúc thư mục:** Tạo các thư mục `output/pdf_modern` và các thư mục con cho fonts, charts.
3. **Tải font:** Tự động tải và giải nén font DejaVu Sans, là font hỗ trợ tiếng Việt có dấu rất tốt, từ GitHub.

In [ ]:
%%capture

import sys
import os
import importlib
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

import import_default
import import_database
import import_other

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)

from import_default import *
from import_database import *
from import_other import *

In [ ]:
# [1.1] Thiết lập cấu trúc thư mục
base_dir = "../output/pdf"
charts_dir = os.path.join(base_dir, "charts")
fonts_dir = os.path.join(base_dir, "fonts")
os.makedirs(charts_dir, exist_ok=True)
os.makedirs(fonts_dir, exist_ok=True)

# [1.2] Hàm tải font hỗ trợ tiếng Việt
def download_vietnamese_fonts():
    """Tải và giải nén font DejaVu Sans từ GitHub để hỗ trợ tiếng Việt."""
    font_zip_url = 'https://github.com/dejavu-fonts/dejavu-fonts/releases/download/version_2_37/dejavu-fonts-ttf-2.37.zip'
    font_files_to_extract = {
        'dejavu-fonts-ttf-2.37/ttf/DejaVuSans.ttf': 'DejaVuSans.ttf',
        'dejavu-fonts-ttf-2.37/ttf/DejaVuSans-Bold.ttf': 'DejaVuSans-Bold.ttf'
    }
    
    try:
        response = requests.get(font_zip_url, timeout=60)
        response.raise_for_status()
        with zipfile.ZipFile(io.BytesIO(response.content)) as zf:
            for path_in_zip, target_name in font_files_to_extract.items():
                target_path = os.path.join(fonts_dir, target_name)
                if not os.path.exists(target_path):
                    with open(target_path, 'wb') as f_out:
                        f_out.write(zf.read(path_in_zip))
                else:
                    print(f"   📁 Font đã tồn tại: {target_name}")
    except Exception as e:
        print(f"❌ Lỗi khi tải font: {e}")
        return False
    return True

# Thực thi tải font
download_vietnamese_fonts()

### Cell 2: Tạo Dữ Liệu và Vẽ Biểu Đồ
Cell này chịu trách nhiệm:
1. **Tạo dữ liệu mẫu (sample data):** Sử dụng `pandas` để tạo các DataFrame về thị trường chứng khoán.
2. **Vẽ biểu đồ:** Sử dụng `plotly` để tạo 3 loại biểu đồ (đường, cột, tròn) với tiêu đề và nhãn tiếng Việt.
3. **Lưu biểu đồ:** Lưu các biểu đồ đã tạo ra thành file ảnh PNG vào thư mục `charts`.

In [ ]:
# [2.1] Hàm tạo dữ liệu mẫu
def create_sample_data():
    """Tạo dữ liệu mẫu cho báo cáo với tiếng Việt có dấu."""
    market_overview = {
        'Chỉ số': ['VN-Index', 'HNX-Index', 'UPCoM-Index'],
        'Giá trị hiện tại': [1284.56, 235.78, 95.42],
        'Thay đổi (%)': [2.34, -0.56, 1.87],
        'Khối lượng (triệu CP)': [425.8, 89.3, 15.7],
        'Giá trị (tỷ VNĐ)': [18247, 1356, 187]
    }
    top_stocks = {
        'Mã cổ phiếu': ['VCB', 'VIC', 'VHM', 'HPG', 'MSN', 'TCB', 'BID', 'GAS'],
        'Tên công ty': [
            'Ngân hàng Ngoại thương Việt Nam', 'Tập đoàn Vingroup',
            'Công ty Cổ phần Vinhomes', 'Tập đoàn Hòa Phát', 'Tập đoàn Masan',
            'Ngân hàng Kỹ thương Việt Nam', 'Ngân hàng Đầu tư và Phát triển',
            'Tập đoàn Dầu khí Việt Nam'
        ],
        'Giá (VNĐ)': [118000, 95000, 88000, 35000, 128000, 28500, 52000, 115000],
        'Vốn hóa (tỷ VNĐ)': [925, 456, 387, 198, 285, 165, 287, 425],
        'Ngành': ['Ngân hàng', 'Bất động sản', 'Bất động sản', 'Thép', 'Thực phẩm', 'Ngân hàng', 'Ngân hàng', 'Năng lượng']
    }
    return {
        'market_overview': pd.DataFrame(market_overview),
        'top_stocks': pd.DataFrame(top_stocks)
    }

# [2.2] Hàm tạo và lưu biểu đồ
def create_and_save_charts():
    """Tạo các biểu đồ Plotly với tiếng Việt và lưu dưới dạng PNG."""
    df_line = pd.DataFrame({
        'Ngày': pd.to_datetime(['2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01']),
        'VIC': [75, 78, 82, 85], 'VCB': [98, 100, 105, 102], 'HPG': [25, 26, 28, 27]
    })
    df_bar_pie = data['top_stocks']

    # Biểu đồ đường
    fig_line = px.line(df_line, x='Ngày', y=['VIC', 'VCB', 'HPG'], title='Diễn Biến Giá Cổ Phiếu')
    
    # Biểu đồ cột
    fig_bar = px.bar(df_bar_pie, x='Mã cổ phiếu', y='Vốn hóa (tỷ VNĐ)', color='Ngành', title='Vốn Hóa Theo Ngành')
    
    # Biểu đồ tròn
    fig_pie = px.pie(df_bar_pie, values='Vốn hóa (tỷ VNĐ)', names='Ngành', title='Tỷ Trọng Vốn Hóa Ngành')

    chart_paths = {
        'line': os.path.join(charts_dir, 'line_chart.png'),
        'bar': os.path.join(charts_dir, 'bar_chart.png'),
        'pie': os.path.join(charts_dir, 'pie_chart.png')
    }

    for fig, path in zip([fig_line, fig_bar, fig_pie], chart_paths.values()):
        fig.update_layout(font_family="DejaVu Sans", title_x=0.5)
        fig.write_image(path, width=800, height=450, scale=4)
        
    return chart_paths

# Thực thi tạo dữ liệu và biểu đồ
data = create_sample_data()
chart_image_paths = create_and_save_charts()


### Cell 3: Dựng và Xuất File PDF
Đây là cell cuối cùng, nơi mọi thứ được kết hợp lại:
1. **Import thư viện PDF:** `fpdf`, `datetime`.
2. **Định nghĩa lớp `VietnamesePDF`:** Một lớp kế thừa từ `FPDF`, được tùy chỉnh để:
   - Tự động nạp font tiếng Việt đã tải.
   - Có các phương thức vẽ header, footer, card, và tiêu đề sección với định dạng chuyên nghiệp.
3. **Hàm `create_vietnamese_report`:** Sắp xếp bố cục, chèn văn bản, bảng biểu, và các hình ảnh biểu đồ vào file PDF.
4. **Xuất file:** Gọi hàm để tạo và lưu file `bao_cao_simplified.pdf`.

In [ ]:
from fpdf import FPDF
from fpdf.enums import XPos, YPos
from datetime import datetime

COLORS = {'primary': '#2E86AB', 'accent': '#F18F01', 'text_dark': '#2C3E50', 'text_light': '#6C757D', 'white': '#FFFFFF', 'border': '#DEE2E6'}

class VietnamesePDF(FPDF):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.font_family = 'DejaVuSans'
        try:
            self.add_font(self.font_family, '', os.path.join(fonts_dir, 'DejaVuSans.ttf'))
            self.add_font(self.font_family, 'B', os.path.join(fonts_dir, 'DejaVuSans-Bold.ttf'))
        except Exception as e:
            print(f"❌ Lỗi nạp font, sẽ dùng font cơ bản: {e}")
            self.font_family = 'Arial'

    def hex_to_rgb(self, hex_color):
        hex_color = hex_color.lstrip('#')
        return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

    def header(self):
        if self.page_no() == 1: return
        self.set_font(self.font_family, 'B', 14)
        self.set_text_color(*self.hex_to_rgb(COLORS['primary']))
        self.cell(0, 10, 'Báo Cáo Phân Tích Thị Trường', 0, 1, 'C')
        self.ln(5)

    def footer(self):
        self.set_y(-15)
        self.set_font(self.font_family, '', 8)
        self.set_text_color(128)
        self.cell(0, 10, f'Trang {self.page_no()}/{{nb}}', 0, 0, 'C')

    def add_section_header(self, title):
        self.ln(8)
        self.set_font(self.font_family, 'B', 16)
        self.set_text_color(*self.hex_to_rgb(COLORS['primary']))
        self.cell(0, 10, title, 0, 1, 'L')
        self.set_draw_color(*self.hex_to_rgb(COLORS['accent']))
        self.line(self.get_x(), self.get_y(), self.w - self.r_margin, self.get_y())
        self.ln(5)

def create_vietnamese_report(data, chart_paths):
    """Tạo báo cáo PDF hiện đại với tiếng Việt có dấu."""
    pdf = VietnamesePDF('P', 'mm', 'A4')
    pdf.alias_nb_pages()

    # Trang bìa
    pdf.add_page()
    pdf.set_font(pdf.font_family, 'B', 32)
    pdf.set_y(80)
    pdf.multi_cell(0, 15, 'BÁO CÁO THỊ TRƯỜNG\nCHỨNG KHOÁN VIỆT NAM', align='C')
    pdf.set_font(pdf.font_family, '', 16)
    pdf.set_y(140)
    pdf.cell(0, 10, f'Báo cáo tháng {datetime.now().month}/{datetime.now().year}', align='C')

    # Trang dữ liệu và biểu đồ
    pdf.add_page()
    pdf.add_section_header('Tổng Quan Thị Trường')
    df_market = data['market_overview']
    pdf.set_font(pdf.font_family, '', 10)
    for i, row in df_market.iterrows():
        pdf.cell(0, 7, f"- {row['Chỉ số']}: {row['Giá trị hiện tại']:,} điểm ({row['Thay đổi (%)']:+.2f}%) - GTGD: {row['Giá trị (tỷ VNĐ)']} tỷ VNĐ", 0, 1)

    pdf.add_section_header('Top Cổ Phiếu Vốn Hóa')
    df_top = data['top_stocks']
    headers = ['Mã CP', 'Tên Công Ty', 'Ngành', 'Vốn Hóa (tỷ)']
    widths = [20, 80, 40, 35]
    pdf.set_font(pdf.font_family, 'B', 10)
    pdf.set_fill_color(*pdf.hex_to_rgb(COLORS['primary']))
    pdf.set_text_color(*pdf.hex_to_rgb(COLORS['white']))
    for header, width in zip(headers, widths):
        pdf.cell(width, 8, header, 1, 0, 'C', True)
    pdf.ln()

    pdf.set_font(pdf.font_family, '', 9)
    pdf.set_text_color(*pdf.hex_to_rgb(COLORS['text_dark']))
    for i, row in df_top.iterrows():
        pdf.cell(widths[0], 7, row['Mã cổ phiếu'], 1)
        pdf.cell(widths[1], 7, row['Tên công ty'], 1)
        pdf.cell(widths[2], 7, row['Ngành'], 1)
        pdf.cell(widths[3], 7, f"{row['Vốn hóa (tỷ VNĐ)']:,}", 1, 1, 'R')
        
    # Chèn biểu đồ
    pdf.add_page()
    pdf.add_section_header('Phân Tích Trực Quan')
    pdf.image(chart_paths['line'], x=10, y=pdf.get_y(), w=190)
    pdf.image(chart_paths['bar'], x=10, y=pdf.get_y()+85, w=190)

    # Lưu file
    output_path = os.path.join(base_dir, "bao_cao_simplified.pdf")
    pdf.output(output_path)
    print("\n" + "="*60)
    print("🎉 TẠO BÁO CÁO PDF ĐƠN GIẢN HÓA THÀNH CÔNG! 🎉")
    print(f"📄 File đã được lưu tại: {os.path.abspath(output_path)}")
    print("="*60)
    return output_path

# Thực thi tạo báo cáo
create_vietnamese_report(data, chart_image_paths)